https://github.com/ThomasDelteil/DeepLearningWithMXNetGluon/blob/master/9_object_segmentation.ipynb

In [1]:
!pip install -U mxnet-cu101==1.7.0
!pip install d2l==0.16.0

     |███████████████████████████████▌| 834.1MB 1.2MB/s eta 0:00:10tcmalloc: large alloc 1147494400 bytes == 0x65116000 @  0x7efccb5c4615 0x591e47 0x4cc179 0x4cc2db 0x50a1cc 0x50beb4 0x507be4 0x509900 0x50a2fd 0x50beb4 0x507be4 0x509900 0x50a2fd 0x50cc96 0x58e683 0x50c127 0x58e683 0x50c127 0x58e683 0x50c127 0x58e683 0x50c127 0x5095c8 0x50a2fd 0x50beb4 0x507be4 0x509900 0x50a2fd 0x50beb4 0x5095c8 0x50a2fd
     |████████████████████████████████| 846.0MB 19kB/s 
  Found existing installation: graphviz 0.10.1
    Uninstalling graphviz-0.10.1:
      Successfully uninstalled graphviz-0.10.1
     |████████████████████████████████| 81kB 4.4MB/s 


In [2]:
!pip install gluoncv

     |████████████████████████████████| 1.0MB 6.4MB/s 
     |████████████████████████████████| 11.8MB 13.6MB/s 
     |████████████████████████████████| 317kB 47.7MB/s 
     |████████████████████████████████| 256kB 34.4MB/s 
     |████████████████████████████████| 102kB 6.4MB/s 
     |████████████████████████████████| 102kB 6.2MB/s 
     |████████████████████████████████| 215kB 46.3MB/s 
     |████████████████████████████████| 675kB 42.2MB/s 
     |████████████████████████████████| 972kB 41.9MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 7.2MB 39.5MB/s 
     |████████████████████████████████| 71kB 8.6MB/s 
     |████████████████████████████████| 962kB 37.1MB/s 
     |████████████████████████████████| 2.6MB 40.1MB/s 
     |████████████████████████████████| 71kB 8.3MB/s 
     |████████████████████████████████| 102kB 12.5MB/s 
  Created wheel for ConfigSpace: filenam

In [3]:
%matplotlib inline
from matplotlib import pyplot as plt
import matplotlib.image as mpimg

import random, multiprocessing, time

import mxnet as mx
from mxnet import image
from mxnet.gluon.data.vision import transforms
from mxnet import gluon, autograd
from mxnet.gluon import nn

from mxboard import SummaryWriter

import numpy as np

import gluoncv
from gluoncv.utils.parallel import *
from gluoncv.utils.viz import get_color_pallete, DeNormalize

ModuleNotFoundError: ignored

In [4]:
!pip install mxboard

     |████████████████████████████████| 51kB 3.0MB/s 


In [5]:
%matplotlib inline
from matplotlib import pyplot as plt
import matplotlib.image as mpimg

import random, multiprocessing, time

import mxnet as mx
from mxnet import image
from mxnet.gluon.data.vision import transforms
from mxnet import gluon, autograd
from mxnet.gluon import nn

from mxboard import SummaryWriter

import numpy as np

import gluoncv
from gluoncv.utils.parallel import *
from gluoncv.utils.viz import get_color_pallete, DeNormalize

In [6]:
nclasses = len(gluoncv.data.VOCSegmentation().classes)

OSError: ignored

In [7]:
nclasses = len(gluoncv.data.VOCSegmentation().classes)

OSError: ignored

In [8]:
nclasses = len(gluoncv.data.VOCDetection().classes)

OSError: ignored

In [9]:
gluoncv.data.VOCDetection()

OSError: ignored

In [ ]:
class FCN(nn.HybridBlock):
    def __init__(self, resnet_backbone, nclass):
        super(FCN, self).__init__()
        self.backbone = resnet_backbone 
        in_channels = 2048
        with self.name_scope():
            self.head = nn.HybridSequential()
            inter_channels = in_channels // 4
            self.head.add(nn.Conv2D(in_channels=in_channels, channels=inter_channels,
                                     kernel_size=3, padding=1))
            self.head.add(nn.BatchNorm(in_channels=inter_channels))
            self.head.add(nn.Activation('relu'))
            self.head.add(nn.Dropout(0.1))
            self.head.add(nn.Conv2D(in_channels=inter_channels, channels=nclass,
                                     kernel_size=1))
        
    def hybrid_forward(self, F, x):
        # run through backbone
        x = self.backbone.conv1(x)
        x = self.backbone.bn1(x)
        x = self.backbone.relu(x)
        x = self.backbone.maxpool(x)
        x = self.backbone.layer1(x)
        x = self.backbone.layer2(x)
        c3 = self.backbone.layer3(x)
        c4 = self.backbone.layer4(c3)

        #run through head
        y = self.head(c4)

        #resize
        y = F.contrib.BilinearResize2D(y, height=480,width=480)
        return y

In [ ]:
ctx_list = [mx.gpu(i) for i in range(4)]
backbone = gluoncv.model_zoo.resnet50_v1b(pretrained=True, dilated=True, ctx=ctx_list)
del backbone.fc

net = FCN(backbone, nclasses)

# Initialize the head
net.head.load_parameters().setattr('lr_mult', 10)
net.head.initialize(mx.init.Xavier(), ctx=ctx_list)